# Разведывательный анализ данных

Определив бизнес-задачу, вы перешли к сбору данных для обучения модели. После нескольких недель парсинга сайта WineEnthusiast вам удалось собрать около 130 тысяч строк обзоров вин для анализа и обучения.

Вот какие признаки вам удалось собрать:

* country — страна-производитель вина.
* description — подробное описание.
* designation — название виноградника, где выращивают виноград для вина.
* points — баллы, которыми WineEnthusiast оценил вино по шкале от 1 до 100.
* price — стоимость бутылки вина.
* province — провинция или штат.
* region_1 — винодельческий район в провинции или штате (например Напа).
* region_2 — конкретный регион. Иногда в пределах винодельческой зоны указываются более конкретные регионы (например Резерфорд в долине Напа), но это значение может быть пустым.
* taster_name — имя сомелье.
* taster_twitter_handle — твиттер сомелье.
* title — название вина, которое часто содержит год и другую подробную информацию.
* variety — сорт винограда, из которого изготовлено вино (например Пино Нуар).
* winery — винодельня, которая производила вино.

In [61]:
import pandas as pd

data = pd.read_csv('data/wine.csv')
wine_data = data.copy()

display(wine_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


None

Задание 4.1
Сколько всего дегустаторов приняло участие в винных обзорах?

In [62]:
display(wine_data['taster_name'].value_counts().count())

19

Задание 4.2
Какова максимальная цена за бутылку в этом наборе данных? Введите целое число.

In [63]:
display(wine_data['price'].nlargest(1).values[0])

3300.0

Задание 5.3

Проанализируйте датасет на наличие дублирующихся винных обзоров. Если дублирующиеся записи есть, удалите их.
В поле ниже введите, сколько дублирующихся записей вам удалось обнаружить.

In [64]:
dupl_columns = list(wine_data.columns)

#ищем дубликаты
mask = wine_data.duplicated(subset=dupl_columns)
data_duplicates = wine_data[mask]
print(f'Число найденных дубликатов: {data_duplicates.shape[0]}')


#удаляем дубликаты
data_dedupped = wine_data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {data_dedupped.shape[0]}')

Число найденных дубликатов: 9983
Результирующее число записей: 119988


In [65]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


None

# Очистка данных

Процент пропущенных данных в столбцах датасета

In [66]:
cols_null_percent = wine_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

region_2                 61.136715
designation              28.825661
taster_twitter_handle    24.015357
taster_name              20.192197
region_1                 16.347493
price                     6.921544
country                   0.048472
province                  0.048472
variety                   0.000769
dtype: float64

Заполнение недостающих данных

In [67]:
wine_data['taster_name'] = wine_data['taster_name'].fillna('unknown')
wine_data['taster_twitter_handle'] = wine_data['taster_twitter_handle'].fillna('unknown')
wine_data['designation'] = wine_data['designation'].fillna('unknown')
wine_data['region_1'] = wine_data['region_1'].fillna('unknown')


Удаление записей с большим количеством пропусков

In [68]:
#создаем копию исходной таблицы
drop_data = wine_data.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
drop_data = drop_data.dropna(how='any', axis=0)
#отображаем результирующую долю пропусков
drop_data.isnull().mean()

country                  0.0
description              0.0
designation              0.0
points                   0.0
price                    0.0
province                 0.0
region_1                 0.0
taster_name              0.0
taster_twitter_handle    0.0
title                    0.0
variety                  0.0
winery                   0.0
dtype: float64

In [69]:
print(drop_data.shape)

(120915, 12)


In [70]:
wine_data.to_csv('data/wine_cleaned.csv')